# Czyszczenie danych z folderu z pobranymi wierszami
**Założenia:**
1. Z zadeklarowanej ścieżki z wierszami przerzucamy całość do nowego folderu
2. Każdy wiersz przerzucamy do folderu z autorem
3. Czyścimy plik o:
    - symbole inne niż: ',", :, ;, ,, ., -
    - wiersze z mniejszą liczbą znaków niż 3
    - podwójne znaki specjalne
    - wielokrotne spacje, tabulatory
    - usuwamy nowe linie
    - usuwamy całą linie:
        - z informacją w stylu: "wiersz przełożył"
        - z samymi cyframi lub spacjami
3. Każdy przerzucony wiersz dodajemy do DataFrame:
    - z informacją o autorze, tytule, ścieżce, liczba wersów, średnia długości wersu, 
    - informacja o rymach (abab / abba / aabb / brak)
    - fladze czy proza

In [273]:
import pandas as pd
import os
import io
import random
import time
import re

In [9]:
FOLDER_ROW = '.\data_raw'
FOLDER_CLEAN = '.\data_clean'

In [385]:
#tworzenie folderu jeśli nieistnieje
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

#oczyszczanie url z unicode na polskie znaki
from urllib.parse import unquote

def cleanup(url):
    try:
        return unquote(url, errors='strict')
    except UnicodeDecodeError:
        return unquote(url, encoding='polish-1')
    
#średnia długość listy
def averageLen(lst):
    lengths = [len(i) for i in lst]
    return 0 if len(lengths) == 0 else (float(sum(lengths)) / len(lengths)) 

def aa_ab_test(poem):
    test = poem.split('\n')
    test = [re.sub(r'[^a-zA-ZąĄćĆęĘńŃóÓśŚżŻźŹ]', '', line) for line in test]
    
    try:
        list0=test     #zaczynamy od wersu 0
        list1=test[1:] #zaczynamy od wersu 1
        list2=test[2:] #zaczynamy od wersu 2

        i = 0
        aabb=0
        abab=0

        for j in range(len(list3)):
            if list0[j][-2:] == list1[j][-2:]: aabb=aabb+1
            if list0[j][-2:] == list2[j][-2:]: abab=abab+1
                
        return aabb, abab
    
    except:
        return -1, -1

In [386]:
#na potrzeby testów do pisania kodu do oczyszczania losujemy wiersz
rnd_author = random.choice(os.listdir(FOLDER_ROW))
rnd_poem = random.choice(os.listdir(os.path.join(FOLDER_ROW, rnd_author)))
poem_path = os.path.join(FOLDER_ROW, rnd_author, rnd_poem)

#wczytanie pliku
with open(poem_path, 'r', encoding="utf8") as file:
    poem = file.read()
    
#zapisanie pliku
create_folder(os.path.join(FOLDER_CLEAN,rnd_author))

# open(os.path.join(FOLDER_CLEAN,rnd_author,rnd_poem), "w", encoding="utf-8") as f:
#    f.write(str(poem))

print(poem)

[Za tych — co batem ścięte liście,

za tych — co ptaki z wosku lane,

tych co im krew znużeniem tryśnie

i tych — co wbici cieniem w ścianę

i za zwierzęta konające

którym powoli oczy bledną —

— chciałbyś odrzucić Bogu — życie,

umrzeć raz drugi jak Zbawiciel.

Ale zawarty tobie upływ

krwi i związane ręce w supły,

bo ty nie twórczym niepokojem,

a tylko, że się patrzeć boisz.
, 
]


In [387]:
print(clean_poem(poem))

Za tych — co batem ścięte liście,
za tych — co ptaki z wosku lane,
tych co im krew znużeniem tryśnie
i tych — co wbici cieniem w ścianę
i za zwierzęta konające
którym powoli oczy bledną —
— chciałbyś odrzucić Bogu — życie,
umrzeć raz drugi jak Zbawiciel.
Ale zawarty tobie upływ
krwi i związane ręce w supły,
bo ty nie twórczym niepokojem,
a tylko, że się patrzeć boisz


In [388]:
def clean_poem(poem):
    clean_poem =''
    #usuń dziwne znaczki:
    poem = re.sub("[@#$%^&*(){}„”“»«/<>|`~=_+]", "", poem) #usuń znaki specjalne
    poem = poem.replace('[','').replace(']','') #usuń znaki []
    poem = poem.replace('\xa0','') #usuń tabulatory
    poem = re.sub(' +', ' ', poem) #usuń wielokrotne spacje
    
    #przechodzimy linijka po linijce w stringu
    for line in poem.split('\n'):
        if line[0:2]==', ':
            line = line[2:]
        if line[0:1]==' ':
            line = line[1:]
        if 'Uwaga! Tekst niniejszy' in line: #Uwaga! Tekst niniejszy w języku polskim został opublikowany w ok. 1860.
            line = ''
        if 'Stosowane słownictwo i ortografia' in line: #Stosowane słownictwo i ortografia pochodzą z tej epoki, prosimy nie nanosić poprawek niezgodnych ze źródłem!
            line = ''
        if line==re.sub('[^0-9 ]', '', line): #gdy same cyfry to usuń
            line = ''
        if len(line.replace(" ", ""))>2:
            clean_poem=clean_poem+line+'\n'
    
    #jeśli tekst kończy się nową linią to ją usuń
    if clean_poem[-1:]=='\n':
        clean_poem=clean_poem[:-2]
    return clean_poem


## Odpalenie na całości

In [230]:
import glob

In [231]:
file_list = []

for filepath in glob.iglob(r'E:\DeepArt\data_raw\**\*.txt', recursive=True):
    file_list.append(filepath)
    
print(f'Mam {len(file_list)} poematów!')

Mam 16308 poematów!


In [395]:
##test

In [394]:
df_poems = pd.DataFrame(columns=['autor','title','path', 'liczba_wersow', 'avg_dlugosc_wersu'])

i = 0

for autor_folder in glob.glob(r'E:\DeepArt\data_raw_test\*'):
    autor = autor_folder.replace('E:\DeepArt\data_raw_test\\','')
    
    for poem_path in glob.glob(os.path.join('E:\\DeepArt\\data_raw_test\\',autor)+'\*'):
        title=poem_path.replace(autor_folder+'\\','')
        
        #wczytanie pliku
        with open(poem_path, 'r', encoding="utf8") as file:
            poem = file.read()
        
        #czyszczenie pliku
        poem_clean = clean_poem(poem)
        
        #tworzenie nowego folderu jeśli nie istnieje (bez dziwnych znaków z url)
        create_folder(os.path.join(FOLDER_CLEAN, cleanup(autor)))
        
        #zapisanie pliku (bez dziwnych znaków z url)
        clean_path = os.path.join(FOLDER_CLEAN, cleanup(autor))+'\\'+cleanup(title)
        with io.open(clean_path, "w", encoding="utf-8") as f:
            f.write(str(poem_clean))
            
        df_poems.loc[i]=[cleanup(autor),
                         cleanup(title),
                         clean_path,
                         len(poem_clean.split('\n')),
                         averageLen(poem_clean.split('\n'))
                        ]
                
        i=i+1
        if i % 500 == 0: print(i)

In [397]:
## całość
df_poems = pd.DataFrame(columns=['autor','title','path', 'liczba_wersow', 'avg_dlugosc_wersu'])

i = 0

for autor_folder in glob.glob(r'E:\DeepArt\data_raw\*'):
    autor = autor_folder.replace('E:\DeepArt\data_raw\\','')
    
    for poem_path in glob.glob(os.path.join('E:\\DeepArt\\data_raw\\',autor)+'\*'):
        title=poem_path.replace(autor_folder+'\\','')
        
        #wczytanie pliku
        with open(poem_path, 'r', encoding="utf8") as file:
            poem = file.read()
        
        #czyszczenie pliku
        poem_clean = clean_poem(poem)
        
        #tworzenie nowego folderu jeśli nie istnieje (bez dziwnych znaków z url)
        create_folder(os.path.join(FOLDER_CLEAN, cleanup(autor)))
        
        #zapisanie pliku (bez dziwnych znaków z url)
        clean_path = os.path.join(FOLDER_CLEAN, cleanup(autor))+'\\'+cleanup(title)
        
        try:
            with io.open(clean_path, "w", encoding="utf-8") as f:
                f.write(str(poem_clean))

            df_poems.loc[i]=[cleanup(autor),
                             cleanup(title),
                             clean_path,
                             len(poem_clean.split('\n')),
                             averageLen(poem_clean.split('\n'))
                            ]
        except:
            print(f'Nieudane dla: {autor}, {title}')
        i=i+1
        if i % 500 == 0: print(i)

Nieudane dla: Adam_Mickiewicz, Co_to_jest%3F.txt
Nieudane dla: Aleksander_Fredro, Co_b%C4%99dzie%3F.txt
500
1000
Nieudane dla: Antoni_Kucharczyk, Czemu%3F_(Kucharczyk).txt
Nieudane dla: Antoni_Kucharczyk, Czy_ci_nie_%C5%BCal%3F.txt
Nieudane dla: Antoni_Kucharczyk, Czy_j%C4%85_znacie%3F.txt
Nieudane dla: Antoni_Kucharczyk, Czy_kochacie%3F.txt
Nieudane dla: Antoni_Kucharczyk, Czy_ty_kochasz%3F.txt
Nieudane dla: Antoni_Kucharczyk, Gdzie_si%C4%99_podzia%C5%82y%3F.txt
Nieudane dla: Antoni_Kucharczyk, Gdzie_szcz%C4%99%C5%9Bcie%3F_(Kucharczyk).txt
Nieudane dla: Antoni_Kucharczyk, Kiedy%C5%BC%3F_(Kucharczyk,_1914).txt
Nieudane dla: Antoni_Kucharczyk, Kto_jest_tw%C3%B3rc%C4%85_litanii_do_Naj%C5%9Bwi%C4%99tszej_Panny_Maryi%3F.txt
Nieudane dla: Antoni_Kucharczyk, O_co%3F.txt
1500
2000
Nieudane dla: Artur_Oppman, Kto_winniejszy%3F.txt
2500
Nieudane dla: Bruno_Jasie%C5%84ski, Prolog_do_%22Foot-Ballu_Wszystkich_%C5%9Awi%C4%99tych%22.txt
Nieudane dla: Bruno_Jasie%C5%84ski, Prolog_do_%22Wesela%22_Stan

12500
Nieudane dla: Stanis%C5%82aw_Grochowiak, Dobranoc_g%C5%82owo_%E2%80%94_jak%C5%BCe%C5%9B_wygl%C4%85da%C5%82a%3F....txt
Nieudane dla: Stanis%C5%82aw_Korab-Brzozowski, %3F.txt
Nieudane dla: Stanis%C5%82aw_Przybyszewski, Gdzie_chwyci%C4%87_ci%C4%99%3F.txt
13000
Nieudane dla: Stanis%C5%82aw_Wyspia%C5%84ski, Czy_Panu_w_oczy_kiedy_%C5%9Bmier%C4%87_zajrza%C5%82a%3F....txt
Nieudane dla: Stanis%C5%82aw_Wyspia%C5%84ski, Czy_wiadomo%C5%9Bci_Pan_chce,_pozytywnych%3F....txt
Nieudane dla: Stanis%C5%82aw_Wyspia%C5%84ski, Wiszary%3F_znany_wyraz,_powszechnie_zu%C5%BCyty.txt
Nieudane dla: Tadeusz_Boy-%C5%BBele%C5%84ski, Jak_sko%C5%84czy%C4%87_z_piek%C5%82em_kobiet%3F.txt
13500
Nieudane dla: Tomasz_Kajetan_W%C4%99gierski, Do_J%C3%B3zefa_Bielawskiego_na_wyraz_%22ono%22_u%C5%BCyty_w_jego_wierszu_%22Do_kr%C3%B3la%22.txt
Nieudane dla: W%C5%82adys%C5%82aw_Be%C5%82za, Co_kocha%C4%87%3F_(1901).txt
Nieudane dla: W%C5%82adys%C5%82aw_Be%C5%82za, Co_kocha%C4%87%3F_(1912).txt
Nieudane dla: W%C5%82adys%C5%82aw_B

In [314]:
df_poems.head()

,autor,title,path,liczba_wersow,avg_dlugosc_wersu
0,Łukasz_Opaliński,"Poeta_Łukasza_Opalinskiego,_Marſzałka_Nadworne...",.\data_clean\Łukasz_Opaliński\Poeta_Łukasza_Op...,14,19.785714
1,Światopełk_Karpiński,Stężała_dłoń_i_pióro....txt,.\data_clean\Światopełk_Karpiński\Stężała_dłoń...,15,40.733333
2,Adam_Asnyk,Ad_Q._Dellium.txt,.\data_clean\Adam_Asnyk\Ad_Q._Dellium.txt,28,34.678571
3,Adam_Asnyk,Ad_Venerem.txt,.\data_clean\Adam_Asnyk\Ad_Venerem.txt,8,35.000000
4,Adam_Asnyk,"Boska_komedya_(Dante,_1895) Część_I.txt",".\data_clean\Adam_Asnyk\Boska_komedya_(Dante,_...",502,39.167331


In [337]:
test

['Pieśń ma zagrzmi i zahuczy',
 'i zatętni wzdłuż i wszerz.',
 'Słuchaj, ludu, i ty, ziemio,',
 'moje słowa w uszy bierz.',
 'Słuchaj, ludu mój, słuchajcie',
 'wy, książęta i królowie:',
 'Ja, ja, matka w Izraelu,',
 'ja zaśpiewam pieśń Jehowie.',
 'Kiedy wyszedł Pan ze Seir,',
 'gdy przez pole Edom szedł.',
 'Ziemia drżała, góry mdlały,',
 'potok Cyton kurczył grzbiet.',
 'Lew w pustyni przypadł k’ziemi',
 'i zakwili jak pacholę:',
 'Kiedy wyszedł Pan ze Seir,',
 'przez edomskie gdy szedł pole,',
 'Z chananejskich wozów konie',
 'lud mój ujrzał śród swych łąk:',
 'Zdarto tarcze z jego ramion,',
 'wytrącono drzewca z rąk.',
 'Po winnicach ptak się zdziwił,',
 'obcej mowy słysząc głosy.',
 'Cudze sierpy żąć poczęły',
 'po pszenicznych łanach kłosy.',
 'Lat dwadzieścia z drżącą wargą',
 'ze krwią w oku patrzał lud,',
 'Jak szerszenie z ust cór jego',
 'wysysały słodki miód.',
 'Spustoszały wsie i grody,',
 'wyschła woda studzien wielu:',
 'Aż powstałam ja, Debora —',
 'wstałam — matka w 

(11, 49)

In [379]:
list0=test #zaczynamy od wersu 0
list1=test[1:] #zaczynamy od wersu 1
list2=test[2:] #zaczynamy od wersu 2

i = 0
aa=0
ab=0

for j in range(len(list3)):
    if list0[j][-2:] == list1[j][-2:]: aa=aa+1
    if list0[j][-2:] == list2[j][-2:]: ab=ab+1

In [347]:
import more_itertools

for w1, w2 in more_itertools.chunked(test, 2):
    print(f'{w1[-3:]} vs {w2[-3:]}')

print('-----------------')
    
for w2, w3 in more_itertools.chunked(test[1:], 2):
    print(f'{w2[-3:]} vs {w3[-3:]}')

czy vs erz
mio vs erz
cie vs wie
elu vs wie
eir vs zed
lay vs iet
emi vs olę
eir vs ole
nie vs hąk
ion vs rąk
iwi vs osy
zęy vs osy
rgą vs lud
ego vs iód
ody vs elu
ora vs elu
ren vs jęk
ków vs zęk
ące vs owę
mną vs owę
ana vs wój
owy vs nój
śno vs icy
mem vs icy
żny vs zed
lay vs iet
urę vs adą
ado vs ado
ory vs zód
yda vs ród
lon vs arę
olu vs are
ski vs Dan
ach vs Pan
roz vs ńce
osy vs ńce
hel vs ódz
ara vs ódz
eka vs cej
eka vs cej
ona vs doń
szy vs roń
żar vs nią
czy vs nią
tka vs rós
aca vs wóz
bne vs szy
ela vs szy
aki vs arz
obą vs asz
aki vs eci
ach vs eci
-----------------
erz vs mio
erz vs cie
wie vs elu
wie vs eir
zed vs lay
iet vs emi
olę vs eir
ole vs nie
hąk vs ion
rąk vs iwi
osy vs zęy
osy vs rgą
lud vs ego
iód vs ody
elu vs ora
elu vs ren
jęk vs ków
zęk vs ące
owę vs mną
owę vs ana
wój vs owy
nój vs śno
icy vs mem
icy vs żny
zed vs lay
iet vs urę
adą vs ado
ado vs ory
zód vs yda
ród vs lon
arę vs olu
are vs ski
Dan vs ach
Pan vs roz
ńce vs osy
ńce vs hel
ódz vs ara
ódz

ValueError: not enough values to unpack (expected 2, got 1)